In [11]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark


# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

In [ ]:
!pip install pyspark

In [ ]:
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [12]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [46]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df= spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df


DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [47]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [48]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg_bed= spark.sql("SELECT ROUND(AVG(price),2), YEAR(date) from home_data where bedrooms == 4 group by YEAR(date) ORDER BY YEAR(date) desc")
avg_bed


DataFrame[round(avg(price), 2): double, year(date): int]

In [49]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_home= spark.sql("SELECT ROUND(AVG(price),2), date_built from home_data where bedrooms == 3 AND bathrooms == 3 group by date_built ORDER BY date_built desc")
avg_home


DataFrame[round(avg(price), 2): double, date_built: string]

In [50]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
avg_home2= spark.sql("SELECT ROUND(AVG(price),2), date_built from home_data where bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000 group by date_built ORDER BY date_built desc")
avg_home2



DataFrame[round(avg(price), 2): double, date_built: string]

In [51]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("SELECT view, ROUND(AVG(price),2) from home_data group by view having ROUND(AVG(price),2) >= 350000").show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  54|           798684.82|
|  69|           750537.94|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|  59|            791453.0|
|  85|          1056336.74|
|  52|           733780.26|
|  71|            775651.1|
|  98|          1053739.33|
|  99|          1061201.42|
|  96|          1017815.92|
| 100|           1026669.5|
|  70|           695865.58|
|  61|           746877.59|
|  75|          1114042.94|
|  78|          1080649.37|
|  89|          1107839.15|
|  77|          1076205.56|
+----+--------------------+
only showing top 20 rows

--- 0.4283297061920166 seconds ---


In [52]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [53]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [54]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from home_data group by view having ROUND(AVG(price),2) < 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           288929.09|
|  15|           284907.04|
|  11|           280356.07|
|  29|           283881.72|
|  42|           289225.45|
|   3|           284314.53|
|  30|           281085.62|
|  34|           286124.07|
|   8|           279099.78|
|  22|           284908.42|
|  28|           285474.25|
|  16|           291990.83|
|  35|           281767.41|
|   0|           285069.21|
|  47|           292925.62|
|  43|           282606.92|
|   5|           278096.94|
|  31|           287988.84|
|  18|           287532.36|
|  27|            281702.6|
+----+--------------------+
only showing top 20 rows

--- 0.39716076850891113 seconds ---


In [55]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.parquet('home_parquet')

In [56]:
# 11. Read the parquet formatted data.
parquet_df= spark.read.parquet('home_parquet')

In [57]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet_temp')

In [59]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from parquet_temp group by view having ROUND(AVG(price),2) < 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           288929.09|
|  15|           284907.04|
|  11|           280356.07|
|  29|           283881.72|
|  42|           289225.45|
|   3|           284314.53|
|  30|           281085.62|
|  34|           286124.07|
|   8|           279099.78|
|  22|           284908.42|
|  28|           285474.25|
|  16|           291990.83|
|  35|           281767.41|
|   0|           285069.21|
|  47|           292925.62|
|  43|           282606.92|
|   5|           278096.94|
|  31|           287988.84|
|  18|           287532.36|
|  27|            281702.6|
+----+--------------------+
only showing top 20 rows

--- 1.4672489166259766 seconds ---


In [61]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")


DataFrame[]

In [62]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False